In [1]:
import scipy.sparse
from sagemaker import image_uris
image_uris.retrieve(framework='tensorflow',region='us-east-1',version='1.12.0',image_scope='inference',instance_type='ml.c5.4xlarge')

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pydantic/_internal/_fields.py:172: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[11/22/24 00:28:05] INFO     Found credentials from IAM Role:                                   ]8;id=718941;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=850625;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


'520713654638.dkr.ecr.us-east-1.amazonaws.com/sagemaker-tensorflow-serving:1.12.0-cpu'

In [2]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 520713654638.dkr.ecr.us-east-1.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [3]:
!docker pull 520713654638.dkr.ecr.us-east-1.amazonaws.com/sagemaker-tensorflow-serving:1.12.0-cpu

1.12.0-cpu: Pulling from sagemaker-tensorflow-serving
Digest: sha256:026c2a51a2e9e258481b17c110f9c9640dc010bf71463d4bd31e925ced7c6a49
Status: Image is up to date for 520713654638.dkr.ecr.us-east-1.amazonaws.com/sagemaker-tensorflow-serving:1.12.0-cpu
520713654638.dkr.ecr.us-east-1.amazonaws.com/sagemaker-tensorflow-serving:1.12.0-cpu


In [4]:
import scipy.sparse
from sagemaker import image_uris
image_uris.retrieve(framework='tensorflow',region='us-east-1',version='2.12.0',image_scope='training',instance_type='ml.c5.4xlarge')

[11/22/24 00:28:15] INFO     Defaulting to only available Python version: py310                   ]8;id=301618;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=376008;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#603\603]8;;\

'763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.12.0-cpu-py310'

In [5]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-east-1.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [7]:
!docker build -t tensorflow-training:2.12.0-cpu-py310-vg -f Dockerfile.train .

[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.0s (1/1) FINISHED                                 docker:default
 => [internal] load build definition from Dockerfile.train                 0.0s
 => => transferring dockerfile: 2B                                         0.0s
ERROR: failed to solve: failed to read dockerfile: open Dockerfile.train: no such file or directory


In [ ]:
!docker images

In [ ]:
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator
import os

role=get_execution_role()

hyperparameters={'epochs': 1}

estimator=Estimator(
    image_uri='tensorflow-training:2.12.0-cpu-py310-vg',
    role=role,
    instance_count=1,
    instance_type='local',
    hyperparameters=hyperparameters,
    output_path='file://{}/data/output'.format(os.getcwd())

)

print('##### ESTIMATOR FIT STARTED')
estimator.fit('file://{}/vgsales.csv'.format(os.getcwd()))
print('##### ESTIMATOR FIT COMPLETED')

In [ ]:
!tar -xzf data/output/output.tar.gz

In [ ]:
!tar -xzf ./data/output/model.tar.gz

In [ ]:
%%sh
# Nome dell'immagine Docker
image_name="tensorflow-training"
echo "image_name: ${image_name} ######################"

# Account AWS corrente
account=$(aws sts get-caller-identity --query Account --output text)
echo "account: ${account} ######################"

# Regione configurata nel profilo AWS attivo
region=$(aws configure get region)
if [ -z "$region" ]; then
  region="us-west-2"  # Regione di default se non specificata
fi
echo "region: ${region} ######################"

# URI completo dell'immagine Docker su ECR
fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image_name}:2.12.0-cpu-py310-vg"
echo "fullname: ${fullname} ######################"

# Creazione del repository ECR (ignora errore se già esistente)
aws ecr create-repository --repository-name "${image_name}" || echo "Repository ${image_name} già esistente"

# Login a ECR
aws ecr get-login-password --region "${region}" | docker login --username AWS --password-stdin "${account}.dkr.ecr.${region}.amazonaws.com"

# Tag dell'immagine Docker con l'URI completo
docker tag "${image_name}:2.12.0-cpu-py310-vg" "${fullname}"

# Push dell'immagine su ECR
docker push "${fullname}"


In [ ]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 737368952498.dkr.ecr.us-east-1.amazonaws.com

In [ ]:
!docker tag tensorflow-training:2.12.0-cpu-py310-vg 339713156346.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.12.0-cpu-py310-vg

In [ ]:
!docker push 339713156346.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.12.0-cpu-py310-vg

In [ ]:
!docker ps

In [ ]:
file_path = '{}/vgsales.csv'.format(os.getcwd())
print(os.path.exists(file_path))


In [ ]:
!ls -l ./data/input/

In [20]:
!ls -l

total 0


In [11]:
import os

# Directory corrente
current_dir = os.getcwd()
print(f"Current directory: {current_dir}")

# Verifica del file
file_path = os.path.join(current_dir, "vgsales.csv")
print(f"File path: {file_path}")
print(f"File exists: {os.path.exists(file_path)}")

# Contenuto della directory
print("Contents of the current directory:")
print(os.listdir(current_dir))


Current directory: /home/ec2-user/SageMaker/.Trash-1000/files/AWSTraining
File path: /home/ec2-user/SageMaker/.Trash-1000/files/AWSTraining/vgsales.csv
File exists: False
Contents of the current directory:
['.ipynb_checkpoints']


In [12]:
!ls -l /home/ec2-user/SageMaker/AWSrepo/AWSTraining/vgsales.csv


ls: cannot access /home/ec2-user/SageMaker/AWSrepo/AWSTraining/vgsales.csv: No such file or directory
